# Sentinel-2 Image Processing

This notebook removes clouds from satellite images (using fmask tif files as input) and gets the median pixel value for all bands across multiple tif files.

## Imports and Setup

In [1]:
import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

import os
import sys
sys.path.insert(0, '../utils')
import geoutils

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

## File Locations

In [3]:
data_dir = "../data/"
sentinel_dir = data_dir + 'Sentinel2/'
cloudmask_dir = data_dir + 'CloudMasks/'
tmp_dir = data_dir + 'tmp/'

if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
tiff_dirs = []
resolutions = ['10', '20', '60']
for resolution in resolutions:
    tiff_dir = sentinel_dir + 'TIFF_{}m/'.format(resolution)
    tiff_dirs.append(tiff_dir)
    if not os.path.exists(tiff_dir):
        os.makedirs(tiff_dir)
tiff_dirs

['../data/Sentinel2/TIFF_10m/',
 '../data/Sentinel2/TIFF_20m/',
 '../data/Sentinel2/TIFF_60m/']

## Get Directories

In [4]:
directory_dict = dict()
for sentinel_folder in os.listdir(sentinel_dir):
    sentinel_name = sentinel_folder.replace('.SAFE', '')
    
    if 'L2A' in sentinel_name:
        directory_dict[sentinel_name] = {}
        directory = sentinel_dir + sentinel_folder + '/GRANULE/'
        
        for resolution in resolutions:
            for folder in os.listdir(directory):
                band_directory = directory + folder + '/IMG_DATA/R{}m/'.format(resolution)
                directory_dict[sentinel_name]['band_locations_{}m'.format(resolution)] = band_directory
                directory_dict[sentinel_name]['bands_{}m'.format(resolution)] = []
                for band in os.listdir(band_directory):
                    directory_dict[sentinel_name]['bands_{}m'.format(resolution)].append(band)
                directory_dict[sentinel_name]['bands_{}m'.format(resolution)].sort()

directory_dict['S2A_MSIL2A_20190325T152641_N0211_R025_T18PYT_20190325T211142']

{'band_locations_10m': '../data/Sentinel2/S2A_MSIL2A_20190325T152641_N0211_R025_T18PYT_20190325T211142.SAFE/GRANULE/L2A_T18PYT_A019613_20190325T152920/IMG_DATA/R10m/',
 'bands_10m': ['T18PYT_20190325T152641_AOT_10m.jp2',
  'T18PYT_20190325T152641_B02_10m.jp2',
  'T18PYT_20190325T152641_B03_10m.jp2',
  'T18PYT_20190325T152641_B04_10m.jp2',
  'T18PYT_20190325T152641_B08_10m.jp2',
  'T18PYT_20190325T152641_TCI_10m.jp2',
  'T18PYT_20190325T152641_WVP_10m.jp2'],
 'band_locations_20m': '../data/Sentinel2/S2A_MSIL2A_20190325T152641_N0211_R025_T18PYT_20190325T211142.SAFE/GRANULE/L2A_T18PYT_A019613_20190325T152920/IMG_DATA/R20m/',
 'bands_20m': ['T18PYT_20190325T152641_AOT_20m.jp2',
  'T18PYT_20190325T152641_B02_20m.jp2',
  'T18PYT_20190325T152641_B03_20m.jp2',
  'T18PYT_20190325T152641_B04_20m.jp2',
  'T18PYT_20190325T152641_B05_20m.jp2',
  'T18PYT_20190325T152641_B06_20m.jp2',
  'T18PYT_20190325T152641_B07_20m.jp2',
  'T18PYT_20190325T152641_B11_20m.jp2',
  'T18PYT_20190325T152641_B12_20m.jp2

## Generate Multiband TIFF File

In [10]:
for resolution, tiff_dir in zip(resolutions, tiff_dirs):
    for image_name in directory_dict:
        print('Processing {}...'.format(image_name))
        image_path = directory_dict[image_name]['band_locations_{}m'.format(resolution)]
        band_files = [image_path + x for x in directory_dict[image_name]['bands_{}m'.format(resolution)]]
        geoutils.combine_bands(
            image_name, 
            band_files, 
            sentinel_dir, 
            output_dir=tiff_dir, 
            plot=False
        )

Processing S2A_MSIL2A_20190325T152641_N0211_R025_T18PYT_20190325T211142...
Processing S2A_MSIL2A_20181225T152631_N0211_R025_T18PYT_20181225T174326...
Processing S2A_MSIL2A_20200208T152631_N0214_R025_T18PYT_20200208T173911...
Processing S2A_MSIL2A_20190114T152641_N0211_R025_T18PYT_20190114T174404...
Processing S2A_MSIL2A_20170723T152641_N9999_R025_T18PYT_20200312T045113...
Processing S2A_MSIL2A_20180320T152641_N9999_R025_T18PYT_20200312T041414...
Processing S2A_MSIL2A_20160618T152642_N9999_R025_T18PYT_20200319T104853...
Processing S2A_MSIL2A_20190703T152641_N0212_R025_T18PYT_20190703T211344...
Processing S2A_MSIL2A_20190325T152641_N0211_R025_T18PYT_20190325T211142...
Processing S2A_MSIL2A_20181225T152631_N0211_R025_T18PYT_20181225T174326...
Processing S2A_MSIL2A_20200208T152631_N0214_R025_T18PYT_20200208T173911...
Processing S2A_MSIL2A_20190114T152641_N0211_R025_T18PYT_20190114T174404...
Processing S2A_MSIL2A_20170723T152641_N9999_R025_T18PYT_20200312T045113...
Processing S2A_MSIL2A_201

### Upload files to GCS
No need to run this cell if the files are already uploaded. 

In [ ]:
#gcs_folders = ['TIFF_10m', 'TIFF_20m', 'TIFF_60m']
#for tiff_dir, gcs_folder in zip(tiff_dirs, gcs_folders):
#    for image_file in os.listdir(tiff_dir):
#        !gsutil -q -m cp {tiff_dir + image_file} gs://immap-sentinel2/{gcs_folder}/

## Median Aggregation and Mosaic Generation

In [5]:
%%time
image_files = [
    'S2A_MSIL2A_20190114T152641_N0211_R025_T18PYT_20190114T174404',
    'S2A_MSIL2A_20190325T152641_N0211_R025_T18PYT_20190325T211142',
    'S2A_MSIL2A_20190703T152641_N0212_R025_T18PYT_20190703T211344'
]
mosaic = geoutils.aggregate(
    image_files, 
    input_dir=tiff_dir[0], 
    cloudmask_dir=cloudmask_dir, 
    tmp_dir=tmp_dir, 
    method='median',
    plot=False
);

100%|██████████| 100/100 [1:01:58<00:00, 37.18s/it]


CPU times: user 19min 1s, sys: 13min 36s, total: 32min 37s
Wall time: 1h 2min 2s
